In [53]:
from tqdm import tqdm
import os
from toolz.curried import *
import requests

In [16]:
from google.colab import drive
MOUNT_POINT = '/content/drive'
drive.mount(MOUNT_POINT, force_remount=True)
TENSORFLOW = os.path.join(MOUNT_POINT,'MyDrive','cse496','Tensorflow')
NOTEBOOK_WS = os.path.join(TENSORFLOW,'workspace','energy-meter-display')
     

Mounted at /content/drive


In [52]:
paths = {
  'workspace': os.path.join(NOTEBOOK_WS),
  'dataset': os.path.join(NOTEBOOK_WS,'dataset'), 
  'labelmap': os.path.join(NOTEBOOK_WS,'annotations','label_map.pbtxt'),
  'annotations': os.path.join(NOTEBOOK_WS,'annotations'),
  'train_record': os.path.join(NOTEBOOK_WS,'annotations','train.record'),
  'test_record': os.path.join(NOTEBOOK_WS,'annotations','test.record'),
  'scripts': os.path.join(TENSORFLOW,'scripts'),
}

Data for: Text detection and Recognition in Raw Image Dataset of Seven Segment Digital Energy Meter Display

https://data.mendeley.com/datasets/fnn44p4mj8

In [10]:
! wget -O dataset.zip https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/fnn44p4mj8-1.zip

--2022-12-04 14:14:01--  https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/fnn44p4mj8-1.zip
Resolving prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)... 52.218.61.184, 52.92.17.90, 52.218.44.170, ...
Connecting to prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com (prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com)|52.218.61.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104185996 (99M) [application/octet-stream]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>]  99.36M  27.6MB/s    in 4.1s    

2022-12-04 14:14:06 (24.0 MB/s) - ‘dataset.zip’ saved [104185996/104185996]



In [11]:
! unzip /content/dataset.zip

Archive:  /content/dataset.zip
  inflating: RAW IMAGES.zip          
  inflating: export-coco-2019-06-11T14_40_00.348423.json  


In [ ]:
!pip install --upgrade numpy


In [6]:
from pathlib import Path
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import skimage.io as io
import pandas as pd 


/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [131]:
coco = COCO(Path('/content/export-coco-2019-06-11T14_40_00.348423.json'))
img_ids = coco.getImgIds()
imgs = coco.loadImgs(img_ids)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [150]:
import xml.etree.cElementTree as ET
from xml.dom import minidom

def prettify(elem):
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

def generate_xml_annotation(filename, im_width, im_height, bboxes):
    try:
        annotation = ET.Element('annotation')
        ET.SubElement(annotation, 'filename').text = filename
        size = ET.SubElement(annotation, 'size')
        ET.SubElement(size, 'width').text = str(im_width)
        ET.SubElement(size, 'height').text = str(im_height)
        ET.SubElement(size, 'depth').text = '3'

        strint = lambda x: str(int(x))

        for index, box in enumerate(bboxes):
            objectBox = ET.SubElement(annotation, 'object')
            ET.SubElement(objectBox, 'name').text = box['name']
            ET.SubElement(objectBox, 'pose').text = 'Unspecified'
            ET.SubElement(objectBox, 'truncated').text = '0'
            ET.SubElement(objectBox, 'difficult').text = '0'
            bndBox = ET.SubElement(objectBox, 'bndbox')
            ET.SubElement(bndBox, 'xmin').text = strint(box['xmin'])
            ET.SubElement(bndBox, 'ymin').text = strint(box['ymin'])
            ET.SubElement(bndBox, 'xmax').text = strint(box['xmax'])
            ET.SubElement(bndBox, 'ymax').text = strint(box['ymax'])

        et = ET.ElementTree(annotation)
        return prettify(annotation)
    except Exception as e:
        print('Error to generate the XML for image {}'.format(filename))
        print(e)

In [151]:
def save_image(url, path):
  img_data = requests.get(url).content
  with open(path, 'wb') as handler:
    handler.write(img_data)

In [154]:
save_path = paths['dataset']

for img in tqdm(imgs):
  filepath = pipe(
      img['file_name'],
      os.path.basename, 
      os.path.splitext,
      first,
      lambda x: os.path.join(save_path, x)
  )
  save_image(img['file_name'], f'{filepath}.jpg'),
  xml_string = pipe(
    img['id'],
    lambda id: coco.getAnnIds(imgIds=[id]),
    coco.loadAnns,
    map(lambda annot: {
        'name': str(annot['category_id']),
        'xmin': annot['bbox'][0], # [x, y ,w ,h]
        'ymin': annot['bbox'][1],
        'xmax': annot['bbox'][0]+annot['bbox'][2],
        'ymax': annot['bbox'][1]+annot['bbox'][3],
    }),
    lambda bboxes: generate_xml_annotation(
      f'{filepath}.xml', 
      img['width'],
      img['height'],
      list(bboxes)
    )
  )
  with open(f'{filepath}.xml', 'w') as file:
    file.write(xml_string)

100%|██████████| 168/168 [00:39<00:00,  4.30it/s]
